# Feature Engineering

## Что такое пропущенные значения
Аналитикам часто приходится работать с данными, в которых есть пропуски. Более того, в реальных данных пропуски сплошь и рядом.

Когда могут появиться пропуски в данных? Например, если мы делаем опрос, люди могут просто не ответить на какие-то из вопросов. В этих местах появляется пропуск. Еще один пример — отправляем данные по протоколу UDP, часть данных теряется — снова пропуски. Ну и самый банальный случай — программа заглючила и не записала часть данных :).

Чтобы программа правильно интерпретировала пропуски, при чтении файла с помощью метода read_csv можно передать в параметр na_values значение или список значений, которые при чтении будут помечены как пропуски.

## Какие бывают пропуски
Вот список значений, которые по умолчанию считаются как пропуски: '', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null'.  

Посмотрите внимательно на список значений выше: можно заметить, что первым значением в списке является пустая строка '' - это именно тот случай, когда значения в данных просто нет.

In [1]:
import pandas as pd

In [2]:
users_df = pd.read_csv('users.csv', sep='\t', encoding='KOI8-R')

In [3]:
users_df.head()

,Юзверь,мейл,Гео
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,NaN


In [4]:
users_df.columns = ['user_id', 'email', 'geo']

In [5]:
users_df.head()

,user_id,email,geo
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,NaN


In [6]:
log_df = pd.read_csv('log.csv', header=None)

In [7]:
log_df.head()

,0,1,2,3
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN


In [8]:
log_df.columns = ['user_id', 'time', 'bet', 'win']

In [9]:
log_df.head()

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN


## Как найти пропуски
В pandas есть метод `isna()`, который возвращает таблицу такой же размерности, что и на вход, но значения в ней - True или False. True, если данное значение является пропуском, и False в ином случае.

In [10]:
log_df.head().isna()

,user_id,time,bet,win
0,False,False,True,True
1,False,False,True,True
2,False,False,True,True
3,False,False,True,True
4,False,False,True,True


Зачем нам смотреть на эту таблицу? На самом деле, незачем, но с помощью нее можно делать полезные вещи. Например, считать количество пропусков и работать с ними, чем мы и займемся дальше.

В numpy пропущенные значения могут быть записаны как специальный объект np.nan, что означает Not a Number.

Проверить на наличие таких значений можно с помощью np.isnan().

P.S. Если вы все-таки хотите посмотреть на красивую (и, возможно, полезную) визуализацию пропущенных значений - обратите внимание на библиотеку [missingno](https://github.com/ResidentMario/missingno).

#### Посчитайте количество пропусков в столбце time. Метод isna() есть не только у DataFrame, но и у Series. Это значит, что применять его можно не только ко всей таблице, но и к каждому столбцу отдельно.

In [11]:
len(log_df[log_df['time'].isna()])

15

## Удаление пропусков
Пропуски можно удалять автоматически. Во многих случаях это правильно, так как данные с большим количеством пропусков часто не имеют смысла и не приносят никакой пользы.  

Удалять данные с пропусками можно с помощью метода `dropna()`.

Параметр `axis` в методе `dropna()` говорит методу, по какой оси удалять значения.  

Если нужно удалить строки, в которых встречается пропуск `(NaN)`, следует указать `axis=0`.  Зачем это делать? Например, у нас из 1000 примеров данных про пользователей пропуски есть в пяти. Разумно их удалить, так как их количество пренебрежимо мало.
Если нужно удалить столбцы, в которых встречается пропуск `(NaN)`, нужно указывать `axis=1`. Зачем? Иногда в одном конкретном столбце пропусков настолько много, что с ними просто не хочется возиться - смысла в них все равно почти нет. 
Еще один интересный параметр - `subset`. Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре `axis` другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск. То же самое работает и наоборот: нужно поменять `axis` на 1 и вместо названий столбцов передавать индексы строк.

Перед удалением строк обязательно сделайте бэкап.

#### Удалите все столбцы, где есть пропуски. Запишите в поле, сколько осталось столбцов в данных после этого.

Используйте оригинальный датасет log.csv.

In [12]:
log_df.dropna(axis=1)

,user_id
0,Запись пользователя № - user_919
1,Запись пользователя № - user_973
2,Запись пользователя № - user_903
3,Запись пользователя № - user_954
4,Запись пользователя № - user_954
...,...
995,Запись пользователя № - user_984
996,#error
997,#error
998,#error


#### Удалите все строки, где есть пропуски. Запишите в поле, сколько осталось строк в данных после этого.

Используйте оригинальный датасет log.csv.

In [13]:
log_df.dropna(subset=['time', 'bet', 'win'])

,user_id,time,bet,win
14,Запись пользователя № - user_917,[2019-01-02 8:57:36,145732.0,1987653.0
29,Запись пользователя № - user_942,[2019-01-04 13:59:42,1678321.0,9876543.0
151,Запись пользователя № - user_982,[2019-01-16 21:54:22,100.0,4749.0
189,Запись пользователя № - user_964,[2019-01-21 18:34:44,200.0,4667.0
205,Запись пользователя № - user_931,[2019-01-22 5:26:59,300.0,4319.0
...,...,...,...,...
967,Запись пользователя № - user_975,[2019-04-19 22:25:15,1000.0,6108.0
971,Запись пользователя № - user_912,[2019-04-20 10:35:49,10554.0,31799.0
972,Запись пользователя № - user_926,[2019-04-20 10:35:50,10354.0,30244.0
976,Запись пользователя № - user_970,[2019-04-20 10:35:54,10354.0,30691.0


Ответ 133

## Как удалить простые дубли
В pandas есть метод для удаления дублей (дубликатов) - drop_duplicates(). Он просто удаляет повторяющиеся строки:

```
import pandas as pd  
df = pd.read_csv('data.csv')
df.drop_duplicates()  
```
У данного метода тоже есть параметр subset, в этом случае нужно передавать список содержащий названия столбцов.  

#### Удалите дубли среди столбцов user_id и time. Запишите в поле ниже, сколько осталось строк после удаления дублей.

Используйте оригинальный датасет log.csv.

In [14]:
log_df.drop_duplicates(subset=['user_id', 'time'])

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
991,Запись пользователя № - user_965,[2019-04-20 12:55:41,800.0,6927.0
992,Запись пользователя № - user_967,[2019-04-20 14:59:36,10154.0,NaN
993,Запись пользователя № - user_973,[2019-04-20 17:09:56,10254.0,NaN
994,Запись пользователя № - user_977,[2019-04-20 18:10:07,10354.0,NaN


Ответ 986

#### Уберите лишний символ, преобразуйте признак time к datetime. После этого найдите наибольшую дату и выведите ее без времени.
Подсказка: можно применить метод max() к получившемуся столбцу со временем.

Не забудьте избавиться от пропусков.

Запишите ответ в формате "YYYY-MM-DD".

In [15]:
def correct_time(time):
    if len(str(time)) > 3:
        return time[1:]
    else:
        return time

log_df['time'] = log_df['time'].apply(correct_time)

log_df['time'] = pd.to_datetime(log_df['time'], format='%Y-%m-%d %H:%M:%S')

In [16]:
log_df['time'].max()

Timestamp('2019-04-20 18:10:07')

Ответ 2019-04-20

## Извлечение признаков времени

- year: возвращает год
- month: возвращает месяц
- day: возвращает день
- hour, minute, second - час, минута, секунда
- dayofweek - день недели, от 0 до 6, где 0 - понедельник, 6 - воскресенье

Кроме них, есть и другие интересные атрибуты, советуем посмотреть [здесь](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html).

In [17]:
log_df['time'].apply(lambda x: x.year)  

0      2019.0
1      2019.0
2      2019.0
3      2019.0
4      2019.0
        ...  
995    2019.0
996       NaN
997       NaN
998       NaN
999       NaN
Name: time, Length: 1000, dtype: float64

Библиотека pandas позволяет использовать аксессор **dt** для упрощения подобной работы:

In [18]:
log_df['time'].dt.year

0      2019.0
1      2019.0
2      2019.0
3      2019.0
4      2019.0
        ...  
995    2019.0
996       NaN
997       NaN
998       NaN
999       NaN
Name: time, Length: 1000, dtype: float64

Аксессор - это атрибут столбца, который хранит переменные типа Timestamp, то есть переменные, которые были строковым представлением времени, а затем изменены с помощью pd.to_datetime(). Если вы попытаетесь обратиться к dt у столбца, в котором лежит что-то отличное от времени, вы получите ошибку.

Чуть больше можно увидеть [здесь](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) (не забудьте посмотреть в исходный код).

#### Найдите минуту, которая встречалась в данных чаще всего. Введите ответ в поле ниже.

In [19]:
log_df['time'].dt.minute.value_counts().index[0]

36.0

Ответ 36

#### Найдите месяц, который встречался в данных реже всего. Введите ответ в поле ниже.

In [20]:
log_df['time'].dt.month.value_counts(ascending=True).index[0]

4.0

Ответ 4

Просто, не так ли?

В реальности мы используем эти же методы для получения более интересных вещей из данных.

Новые интересные знания, которые получаются из данных, мы называем `инсайтами (insights)`.

Следующие пара заданий позволят нам получить результаты, которые можно называть инсайтами и которые могут принести реальную пользу.

#### Посчитайте, сколько дней в данных являются выходными (то есть субботой или воскресеньем). Введите ответ в поле ниже.

In [21]:
log_df['time'].dt.dayofweek.apply(lambda x: 1 if x == 5 or x == 6 else 0).sum()

283

Ответ 283

#### Посчитайте, какое время дня встречается в данных реже всего. Введите ответ в поле ниже: ночь, утро, день или вечер.

Договоримся, что с 0 до 5 часов - ночь, с 6 до 11 - утро, с 12 до 17 - день, с 18 до 23 - вечер.

Важно: для выполнения задания вам нужно будет избавиться от пропусков только в столбце time. Вспомните, как избавиться от пропусков только по конкретному признаку.

In [22]:
def day_period(h):
    if h >= 0 and h <= 5:
        return 'ночь'
    elif h >= 6 and h <= 11:
        return 'утро'
    elif h >= 12 and h <= 17:
        return 'день'
    else:
        return 'вечер'
        

log_df['time'].dropna().dt.hour.apply(day_period).value_counts(ascending=True)

вечер    227
день     240
утро     253
ночь     265
Name: time, dtype: int64

Ответ: вечер

#### Давайте повторим то, что мы прошли в этой секции. Напишите код, который создаст признак hour из признака time в датасете log.csv. Для этого:

1. загрузите датасет log.csv в переменную log, дальше работать будем с ней;

2. установите имена столбцов: ['user_id', 'time', 'bet', 'win'];

3. избавьтесь от пропусков в log;

4. приведите переменную time к подходящему для извлечения признаков виду;

5. получите значение часа для каждой строки в переменной time и запишите в столбец hour в log.

Результатом будет таблица log со столбцом hour внутри.

In [29]:
import pandas as pd
log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id', 'time', 'bet', 'win']
log = log.dropna()
log['time'] = log['time'].apply(lambda x: x[1:])  
log['time'] = pd.to_datetime(log['time'])  
log['hour'] = log['time'].dt.hour
log

,user_id,time,bet,win,hour
14,Запись пользователя № - user_917,2019-01-02 08:57:36,145732.0,1987653.0,8
29,Запись пользователя № - user_942,2019-01-04 13:59:42,1678321.0,9876543.0,13
151,Запись пользователя № - user_982,2019-01-16 21:54:22,100.0,4749.0,21
189,Запись пользователя № - user_964,2019-01-21 18:34:44,200.0,4667.0,18
205,Запись пользователя № - user_931,2019-01-22 05:26:59,300.0,4319.0,5
...,...,...,...,...,...
967,Запись пользователя № - user_975,2019-04-19 22:25:15,1000.0,6108.0,22
971,Запись пользователя № - user_912,2019-04-20 10:35:49,10554.0,31799.0,10
972,Запись пользователя № - user_926,2019-04-20 10:35:50,10354.0,30244.0,10
976,Запись пользователя № - user_970,2019-04-20 10:35:54,10354.0,30691.0,10
